# 영문 이름으로 성별 예측

### NLTK(Natural Language Toolkit) 패키지
- NLTK는 교육용으로 개발된 자연어 처리 및 문서 분석용 파이썬 패키지다. 다양한 기능 및 예제를 가지고 있으며 실무 및 연구에서도
많이 사용된다. NLTK 패키지가 제공하는 주요 기능은 다음과 같다.

 * 말뭉치
 * 토큰 생성
 * 형태소 분석
 * 품사 태깅

- 말뭉치
말뭉치(corpus)는 자연어 분석 작업을 위해 만든 샘플 문서 집합을 말한다. 단순히 소설, 신문 등의 문서를 모아놓은 것도
있지만 품사. 형태소, 등의 보조적 의미를 추가하고 쉬운 분석을 위해 구조적인 형태로 정리해 놓은 것을 포함한다.
NLTK 패키지의 corpus 서브패키지에서는 다양한 연구용 말뭉치를 제공한다. 이 목록은 전체 corpus의 일부일 뿐이다.
말뭉치 자료는 설치시에 제공되지 않고 download 명령으로 사용자가 다운로드 받아야 한다. nltk.download('book')
명령을 실행하면 NLTK 패키지 사용자 설명서에서 요구하는 대부분의 말뭉치를 다운로드 받아준다.

In [1]:
from nltk.corpus import names          # nltk: national language toolkit (자연어처리 패키지)
import nltk
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [2]:
names.words()[:5], len(names.words())

(['Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi'], 7944)

In [3]:
names.words('male.txt')[:5]    # list

['Aamir', 'Aaron', 'Abbey', 'Abbie', 'Abbot']

In [4]:
# 남자 이름과 여자 이름이 데이터베이스가 나뉘어 있음
labeled_names = [(name, '남자') for name in names.words('male.txt')] +   \
                 [(name, '여자') for name in names.words('female.txt')]   # list

labeled_names[:5], labeled_names[-5:]

([('Aamir', '남자'),
  ('Aaron', '남자'),
  ('Abbey', '남자'),
  ('Abbie', '남자'),
  ('Abbot', '남자')],
 [('Zorine', '여자'),
  ('Zsa Zsa', '여자'),
  ('Zsazsa', '여자'),
  ('Zulema', '여자'),
  ('Zuzana', '여자')])

- 앞에는 남자, 뒤에는 여자의 이름이 들어 있다. 데이터 분석에서 랜덤한 성격을 주기 위해서 이들을 랜덤하게 섞는 작업이 필요하다. 이를 위해서 shuffle() 함수를 사용한다.

In [5]:
import random
random.shuffle(labeled_names)
labeled_names[:10]

[('Matthus', '남자'),
 ('Hildegaard', '여자'),
 ('Vaughan', '남자'),
 ('Heloise', '여자'),
 ('Nate', '남자'),
 ('Evania', '여자'),
 ('Timotheus', '남자'),
 ('Meredith', '여자'),
 ('Bamby', '여자'),
 ('Inez', '여자')]

In [6]:
len(labeled_names)

7944

- 이름의 마지막 글자를 얻는 함수 gender_features()라고 만들었다.

In [7]:
# 이름의 마지막 알파벳을 특성으로 사용 (이게 성별과 관련이 많다고 알려짐)
def gender_features(word):
    return {'last_letter': word[-1]}

gender_features('Sopi')

{'last_letter': 'i'}

In [8]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
featuresets[0:10]

[({'last_letter': 's'}, '남자'),
 ({'last_letter': 'd'}, '여자'),
 ({'last_letter': 'n'}, '남자'),
 ({'last_letter': 'e'}, '여자'),
 ({'last_letter': 'e'}, '남자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 's'}, '남자'),
 ({'last_letter': 'h'}, '여자'),
 ({'last_letter': 'y'}, '여자'),
 ({'last_letter': 'z'}, '여자')]

In [9]:
train_set, test_set = featuresets[2000:], featuresets[:2000]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [10]:
classifier.classify(gender_features('Sephipa'))   # predict

'여자'

In [11]:
classifier.classify(gender_features('Chanho'))

'남자'

In [12]:
# 분류 모델이 동작을 할 때 어떤 요소들이 중요한 역할을 했는지를 아래와 같이 알아 볼 수 있다.
# 여기서는 중요한 요소 상위 5개만 출력
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'                여자 : 남자     =     46.4 : 1.0
             last_letter = 'k'                남자 : 여자     =     40.4 : 1.0
             last_letter = 'f'                남자 : 여자     =     11.9 : 1.0
             last_letter = 'd'                남자 : 여자     =     10.9 : 1.0
             last_letter = 'p'                남자 : 여자     =     10.5 : 1.0


In [13]:
print(nltk.classify.accuracy(classifier, train_set))
print(nltk.classify.accuracy(classifier, test_set))

0.7637954239569313
0.76


### 다양한 특성 사용
- 이번에는 새로운 특성을 사용해 본다. 이름의 첫 번째 글자, 이름의 길이, 그리고 각 알파벳이 몇 번 등장했는지를 추가로 특성으로 사용해보겠다.
- features에 다음과 같이 특성 컬럼 들을 추가했다. 이러한 특성을 얻는 함수를 gender_features2라고 정의했다.

In [30]:
def gender_features2(name):
    features = {}                                   # dict
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    features["length"] = len(name)
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
    return features

gender_features2('Josssshua')

{'first_letter': 'j',
 'last_letter': 'a',
 'length': 9,
 'count(a)': 1,
 'count(b)': 0,
 'count(c)': 0,
 'count(d)': 0,
 'count(e)': 0,
 'count(f)': 0,
 'count(g)': 0,
 'count(h)': 1,
 'count(i)': 0,
 'count(j)': 1,
 'count(k)': 0,
 'count(l)': 0,
 'count(m)': 0,
 'count(n)': 0,
 'count(o)': 1,
 'count(p)': 0,
 'count(q)': 0,
 'count(r)': 0,
 'count(s)': 4,
 'count(t)': 0,
 'count(u)': 1,
 'count(v)': 0,
 'count(w)': 0,
 'count(x)': 0,
 'count(y)': 0,
 'count(z)': 0}

In [15]:
labeled_names[0:10]

[('Matthus', '남자'),
 ('Hildegaard', '여자'),
 ('Vaughan', '남자'),
 ('Heloise', '여자'),
 ('Nate', '남자'),
 ('Evania', '여자'),
 ('Timotheus', '남자'),
 ('Meredith', '여자'),
 ('Bamby', '여자'),
 ('Inez', '여자')]

In [31]:
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
featuresets[0]

({'first_letter': 'm',
  'last_letter': 's',
  'length': 7,
  'count(a)': 1,
  'count(b)': 0,
  'count(c)': 0,
  'count(d)': 0,
  'count(e)': 0,
  'count(f)': 0,
  'count(g)': 0,
  'count(h)': 1,
  'count(i)': 0,
  'count(j)': 0,
  'count(k)': 0,
  'count(l)': 0,
  'count(m)': 1,
  'count(n)': 0,
  'count(o)': 0,
  'count(p)': 0,
  'count(q)': 0,
  'count(r)': 0,
  'count(s)': 1,
  'count(t)': 2,
  'count(u)': 1,
  'count(v)': 0,
  'count(w)': 0,
  'count(x)': 0,
  'count(y)': 0,
  'count(z)': 0},
 '남자')

In [17]:
train_set, test_set = featuresets[2000:], featuresets[:2000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.7885


In [18]:
classifier.show_most_informative_features(20)   # last letter 가 가장  중요한 역할을 하는 것을 볼 수 있음

Most Informative Features
             last_letter = 'a'                여자 : 남자     =     46.4 : 1.0
             last_letter = 'k'                남자 : 여자     =     40.4 : 1.0
             last_letter = 'f'                남자 : 여자     =     11.9 : 1.0
             last_letter = 'd'                남자 : 여자     =     10.9 : 1.0
             last_letter = 'p'                남자 : 여자     =     10.5 : 1.0
             last_letter = 'v'                남자 : 여자     =      9.8 : 1.0
             last_letter = 'm'                남자 : 여자     =      9.7 : 1.0
             last_letter = 'o'                남자 : 여자     =      7.6 : 1.0
             last_letter = 'g'                남자 : 여자     =      6.9 : 1.0
                count(v) = 2                  여자 : 남자     =      6.9 : 1.0
             last_letter = 'r'                남자 : 여자     =      6.2 : 1.0
             last_letter = 'z'                남자 : 여자     =      5.6 : 1.0
                count(a) = 3                  여자 : 남자     =      5.2 : 1.0

### 위의 결과를 보면,
- 이번 분류에 가장 영향을 많이 준 요소를 20 개 출력했음.
- 이름의 마지막 글자가 대부분 중요한 역할을 하고 있음 관찰.
- 'a' 가 세 개 포함되면 여성의 비율이 높고, 첫 글자가 'w' 이거나 'w' 의 수가 한 개이면 남성의 비율이 높음.

### 검증 대이터에 대해 성능 평가
- 특성 추출에는 gender_features() 를 사용하고 성능은 아래와 같음.

In [19]:
print(len(labeled_names))
labeled_names[:5]   # list

7944


[('Matthus', '남자'),
 ('Hildegaard', '여자'),
 ('Vaughan', '남자'),
 ('Heloise', '여자'),
 ('Nate', '남자')]

In [20]:
# 검증 데이터에 대해 성능 평가
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

In [21]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))  # validation

0.778


In [22]:
train_set[:20]

[({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'y'}, '여자'),
 ({'last_letter': 'e'}, '남자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'e'}, '여자'),
 ({'last_letter': 's'}, '남자'),
 ({'last_letter': 'n'}, '남자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'n'}, '여자'),
 ({'last_letter': 'e'}, '여자'),
 ({'last_letter': 'e'}, '여자'),
 ({'last_letter': 'e'}, '여자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 't'}, '남자'),
 ({'last_letter': 'e'}, '여자'),
 ({'last_letter': 'a'}, '여자'),
 ({'last_letter': 'e'}, '여자'),
 ({'last_letter': 'y'}, '여자'),
 ({'last_letter': 'i'}, '남자'),
 ({'last_letter': 'a'}, '여자')]

#### 잘못 예측한 내용을 나열해 보자

In [23]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

In [24]:
for (tag, guess, name) in errors[:20]:
    print('correct={:<8s} guess={:<8s} name={:<30}'.format(tag, guess, name))
                                                # < : Left aligned to the remaining space (<, >, ^, =)

correct=남자       guess=여자       name=Tracie                        
correct=여자       guess=남자       name=Gwyn                          
correct=남자       guess=여자       name=Hillary                       
correct=남자       guess=여자       name=Kingsly                       
correct=여자       guess=남자       name=Gates                         
correct=여자       guess=남자       name=Cher                          
correct=여자       guess=남자       name=Dorit                         
correct=여자       guess=남자       name=Piper                         
correct=남자       guess=여자       name=Price                         
correct=여자       guess=남자       name=Michal                        
correct=남자       guess=여자       name=Dougie                        
correct=남자       guess=여자       name=Patty                         
correct=여자       guess=남자       name=Deloris                       
correct=여자       guess=남자       name=Raquel                        
correct=여자       guess=남자       name=Margot     

### 마지막 두 글자 사용

In [25]:
def gender_features(word):
    return {'suffix1': word[-1:],
            'suffix2': word[-2:]}

In [26]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]

In [27]:
train_set[:10]

[({'suffix1': 'a', 'suffix2': 'ha'}, '여자'),
 ({'suffix1': 'y', 'suffix2': 'my'}, '여자'),
 ({'suffix1': 'e', 'suffix2': 'ke'}, '남자'),
 ({'suffix1': 'a', 'suffix2': 'la'}, '여자'),
 ({'suffix1': 'e', 'suffix2': 'ie'}, '여자'),
 ({'suffix1': 's', 'suffix2': 'us'}, '남자'),
 ({'suffix1': 'n', 'suffix2': 'in'}, '남자'),
 ({'suffix1': 'a', 'suffix2': 'ia'}, '여자'),
 ({'suffix1': 'n', 'suffix2': 'nn'}, '여자'),
 ({'suffix1': 'e', 'suffix2': 'ye'}, '여자')]

In [28]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.784


# Exercise

In [29]:
# for exercise
test = [2,3,4,5,2,2,2]
test.count(2)    # 4
test2 = 'abcdefgggggggggggGgggggggggggg'
test2.lower().count('g')    # 24
test2.count('g')  # 23
test2.count('{}'.format('ggg'))       # same, % 연산자 대신 {} 를 써서 format() 사용함. - 새로운 방식

7